<a href="https://colab.research.google.com/github/Yahred/evolutionary-computation/blob/main/CGXV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de rutas a través de algoritmos genéticos

Se cargan módulos necesarios

In [200]:
pip install deap

In [201]:
import json
import math
import random
import folium

import matplotlib.pyplot as plt
import numpy as np

from deap import base, creator, tools, algorithms
from urllib.request import urlopen
from IPython.display import display

Se cargan datos sobre las capitales

In [202]:
path = 'https://raw.githubusercontent.com/Yahred/evolutionary-computation/main/data/capitales.json'
archivo = urlopen(path).read()
data = json.loads(archivo)

CAPITALES = data['capitales']
COORDENADAS = data['coordenadas']
DISTANCIAS = data['distancias']
ESPACIOS = 8
CENTRO_EU = [54.6872, 25.2797]
ZOOM_MAPA = 4

Se calcula la longitud del cromosoma en base a las capitales y el número de espacios disponibles

In [203]:
LONGITUD_CROMOSOMA = 0
for i in range(ESPACIOS):
  LONGITUD_CROMOSOMA += math.ceil(math.log(len(CAPITALES) - i, 2))

LONGITUD_CROMOSOMA

28

In [204]:
CAPITALES

['Madrid',
 'París',
 'Berlín',
 'Roma',
 'Atenas',
 'Lisboa',
 'Varsovia',
 'Praga',
 'Ámsterdam',
 'Viena',
 'Estocolmo',
 'Budapest']

In [205]:
COORDENADAS

{'Madrid': {'latitud': 40.416775, 'longitud': -3.70379},
 'París': {'latitud': 48.856613, 'longitud': 2.352222},
 'Berlín': {'latitud': 52.520008, 'longitud': 13.404954},
 'Roma': {'latitud': 41.902782, 'longitud': 12.496366},
 'Atenas': {'latitud': 37.98381, 'longitud': 23.727539},
 'Lisboa': {'latitud': 38.722252, 'longitud': -9.139337},
 'Varsovia': {'latitud': 52.229676, 'longitud': 21.012229},
 'Praga': {'latitud': 50.075538, 'longitud': 14.4378},
 'Ámsterdam': {'latitud': 52.366696, 'longitud': 4.89454},
 'Viena': {'latitud': 48.208174, 'longitud': 16.373819},
 'Estocolmo': {'latitud': 59.329323, 'longitud': 18.068581},
 'Budapest': {'latitud': 47.497913, 'longitud': 19.040236}}

In [206]:
DISTANCIAS

{'Madrid': {'Madrid': 0,
  'París': 1056,
  'Berlín': 1886,
  'Roma': 2226,
  'Atenas': 2760,
  'Lisboa': 634,
  'Varsovia': 2502,
  'Praga': 1752,
  'Ámsterdam': 1375,
  'Viena': 2275,
  'Estocolmo': 2898,
  'Budapest': 1775},
 'París': {'Madrid': 1056,
  'París': 0,
  'Berlín': 878,
  'Roma': 1105,
  'Atenas': 1842,
  'Lisboa': 1490,
  'Varsovia': 1052,
  'Praga': 1029,
  'Ámsterdam': 431,
  'Viena': 1133,
  'Estocolmo': 1903,
  'Budapest': 1345},
 'Berlín': {'Madrid': 1886,
  'París': 878,
  'Berlín': 0,
  'Roma': 1180,
  'Atenas': 2040,
  'Lisboa': 2154,
  'Varsovia': 265,
  'Praga': 280,
  'Ámsterdam': 657,
  'Viena': 683,
  'Estocolmo': 1618,
  'Budapest': 684},
 'Roma': {'Madrid': 2226,
  'París': 1105,
  'Berlín': 1180,
  'Roma': 0,
  'Atenas': 1584,
  'Lisboa': 1956,
  'Varsovia': 1661,
  'Praga': 1454,
  'Ámsterdam': 1632,
  'Viena': 857,
  'Estocolmo': 2376,
  'Budapest': 1330},
 'Atenas': {'Madrid': 2760,
  'París': 1842,
  'Berlín': 2040,
  'Roma': 1584,
  'Atenas': 0,
  '

Graficamos las capitales que tenemos de opción

In [237]:
def marcar_capitales():
  mapa = folium.Map(location=CENTRO_EU, zoom_start=ZOOM_MAPA)

  for (capital, coord) in COORDENADAS.items():
    lat = coord['latitud']
    lng = coord['longitud']
    folium.Marker([lat, lng], tooltip=capital).add_to(mapa)

  return mapa

marcar_capitales()

# Definición de la función de la evaluación

In [208]:
class Capital:
  def __init__(self, nombre: str, lat, lng) -> None:
    self.nombre = nombre
    self.lat = lat
    self.lng = lng

  def __str__(self):
    return f'{self.nombre} Latitud: {self.lat} Longitud: {self.lng}'

Definimos una función para gráficar el mapa de las capitales

In [234]:
def graficar_capitales(capitales: list[Capital]):
  mapa = marcar_capitales()

  coordenadas = []
  for capital in capitales:
    folium.Marker([capital.lat, capital.lng], tooltip=capital).add_to(mapa)
    coordenadas.append((capital.lat, capital.lng))

  coord = (capitales[0].lat, capitales[0].lng)
  coordenadas.append(coord)

  linea = folium.PolyLine(coordenadas, color='red')
  linea.add_to(mapa)

  return mapa

In [233]:
def capital_factory(nombre: str):
  coordenadas = COORDENADAS[nombre]
  return Capital(nombre, coordenadas['latitud'], coordenadas['longitud'])

In [232]:
def calcular_distancia(a: Capital, b: Capital):
  distancias = DISTANCIAS[a.nombre]
  return distancias[b.nombre]

In [231]:
def calcular_recorrido(capitales: list[Capital]) -> float:
  distancia = 0
  ruta = capitales.copy()

  for i in range(1, len(ruta)):
    a = capitales[i - 1]
    b = capitales[i]
    distancia += calcular_distancia(a, b)

  distancia += calcular_distancia(ruta[-1], ruta[0])

  return distancia

In [230]:
def decode_ind(ind: str) -> list[Capital]:
  opciones = CAPITALES.copy()
  capitales = []
  bin_restante = ''.join([str(gen) for gen in ind])
  for i in range(ESPACIOS):
    n_bits = math.ceil(math.log(len(opciones), 2))
    segmento_bin = bin_restante[:n_bits]
    bin_restante = bin_restante[n_bits:]

    seleccion = int(segmento_bin, 2)

    if seleccion >= len(opciones):
      seleccion = len(opciones) - 1

    capital_seleccionada = capital_factory(opciones.pop(seleccion))
    capitales.append(capital_seleccionada)

  return capitales

In [229]:
def fitness(ind: list[int]):
  capitales = decode_ind(ind)
  distancia = calcular_recorrido(capitales)
  return distancia,

# Definición de los individuos



In [286]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("attr_bool", random.randint, 0, 1)

toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, LONGITUD_CROMOSOMA)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.4)
toolbox.register("select", tools.selTournament, tournsize=5)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)

/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [287]:
ind = toolbox.individual()
ind

[0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1]

In [288]:
random.seed(64)
n_gen = 2000
initial_pop = 200

pop = toolbox.population(n=initial_pop)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.6, mutpb=0.2, ngen=n_gen, halloffame=hof, verbose=True, stats=stats)

ganador = tools.selBest(pop, k=1)[0]

gen	nevals	avg    
0  	200   	11952.4
1  	128   	10503.4
2  	128   	9710.4 
3  	125   	9070.28
4  	140   	9011.33
5  	138   	8151.73
6  	124   	7931.1 
7  	156   	7821.63
8  	140   	7347.01
9  	147   	7525.66
10 	145   	6432.24
11 	137   	6236.6 
12 	136   	6520.27
13 	145   	6210.27
14 	144   	6139.31
15 	131   	6149.06
16 	132   	5903.88
17 	134   	6207.88
18 	143   	5923.23
19 	137   	6072   
20 	138   	5826.47
21 	138   	6062.65
22 	133   	6179.63
23 	136   	5991.31
24 	133   	6438.85
25 	140   	6253.27
26 	118   	6082.83
27 	149   	6190.83
28 	127   	5799.97
29 	139   	5898.26
30 	141   	6008.12
31 	145   	6205.63
32 	133   	6268.06
33 	136   	5887.65
34 	120   	6042.97
35 	148   	6214.51
36 	142   	6508.53
37 	139   	6206.64
38 	143   	6247.91
39 	145   	5978.28
40 	138   	6020.49
41 	156   	6531.05
42 	125   	6218.81
43 	128   	6296.55
44 	135   	5998.58
45 	138   	6101.68
46 	120   	5917.4 
47 	120   	6045.35
48 	139   	6608.2 
49 	142   	6037.66
50 	130   	6150.69
51 	138   	6

In [289]:
distancia, = fitness(ganador)
distancia

4728

In [276]:
capitales = decode_ind(ind)

In [277]:
graficar_capitales(capitales)

# Algoritmo genético compacto

In [264]:
import random

class Individual:
    def __init__(self, chrom: list[int]) -> None:
        self.chrom = chrom
        self.fitness = None

    def __str__(self) -> str:
        return '{} Fitness: {}'.format(self.chrom, self.fitness)


def initialize_probs(num_genes: int) -> list[float]:
    return [0.5 for _ in range(num_genes)]


def create_individual(probs: list[float]):
    chrom = ['1' if random.uniform(0, 1) < prob else '0' for prob in probs]
    return Individual(''.join(chrom))


def compete(a: Individual, b: Individual, fitness: callable, fitness_min: bool):
    a.fitness = fitness(a.chrom)
    b.fitness = fitness(b.chrom)

    if a.fitness < b.fitness and fitness_min:
        return a, b

    if a.fitness > b.fitness and not fitness_min:
        return a, b

    return b, a


def adjust_probs(probs: list[float], winner: Individual, loser: Individual, step: int):
    new_probs = []

    for i in range(len(probs)):
        loser_gen = loser.chrom[i]
        winner_gen = winner.chrom[i]

        if winner_gen == loser_gen:
            new_probs.append(probs[i])
            continue

        if winner_gen == '0':
            new_probs.append(probs[i] - 1 / step)
            continue

        new_probs.append(probs[i] + 1 / step)
    return new_probs


def has_converged(probs: list[float], convergence_criteria: float):
    for prob in probs:
        diff = 1 - prob
        if diff > convergence_criteria:
            return False
    return True


def evolve(fitness: callable, num_genes: int, generations: int, step: int = 0.05, convergence_criteria=0.001, fitness_min=False):
    best = None
    probs = initialize_probs(num_genes)

    for _ in range(generations):
        a = create_individual(probs)
        b = create_individual(probs)

        winner, loser = compete(a, b, fitness, fitness_min)

        if not best:
            best = winner
        elif winner.fitness > best.fitness and not fitness_min:
            best = winner
        elif winner.fitness < best.fitness and fitness_min:
            best = winner

        probs = adjust_probs(probs, winner, loser, step)

        if has_converged(probs, convergence_criteria):
            break

    return best.chrom, best.fitness

Definimos la función de evaluación para el algoritmo genético compacto

In [253]:
def fitness_compact(ind: list[str]):
  capitales = decode_ind(ind)
  distancia = calcular_recorrido(capitales)
  return distancia

Ejecutamos la evolución

In [269]:
generations = 1000

best, distancia = evolve(fitness_compact, LONGITUD_CROMOSOMA, generations, step=0.01, fitness_min=True)

In [272]:
best

'1001011100111101110010111110'

In [271]:
capitales_compacto = decode_ind(best)

calcular_recorrido(capitales_compacto)

8360

In [270]:
graficar_capitales(capitales_compacto)